### 1. Loading libraries and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [10]:
data = pd.read_csv('BlackMonday.csv')
data.head()

,Unnamed: 0,GSPC.Adjusted,time
0,1,5.031744,1984.500000
1,2,5.035003,1984.502732
2,3,5.028868,1984.508197
3,4,5.025458,1984.510929
4,5,5.032788,1984.519126


In [11]:
data = data.iloc[:,1:]

In [12]:
data.head()

,GSPC.Adjusted,time
0,5.031744,1984.500000
1,5.035003,1984.502732
2,5.028868,1984.508197
3,5.025458,1984.510929
4,5.032788,1984.519126


In [98]:
data['GSPC.Adjusted'].max()

5.81940017210343

### 2. Defining equation for the LPPL model

In [18]:
def LPPL(A, B, C1, C2, m, omega, t_c, data):
    price_pred = [A + B*(t_c - data[t])**m + 
                  C1 * ((t_c - data[t])**m) * np.cos(omega * np.log(t_c - data[t])) +
                  C2 * ((t_c - data[t])**m) * np.sin(omega * np.log(t_c - data[t])) 
                  for t in range(data.shape[0])
                 ]
    return price_pred

### 3. Cost function and Jacobian

In [30]:
def CostFunction(price, price_pred):
    cost = (price - price_pred).sum()

In [87]:
def Jacobian(A, B, C1, C2, m, omega, t_c, data):
    grad_t_c = np.array([-(B*m*(t_c - data[t])**(m-1) + (C1 * m * (t_c - data[t])**m) * np.cos(omega * np.log(t_c - data[t])) -
                  (C1*omega*(t_c - data[t])**m) * np.sin(omega * np.log(t_c - data[t])) +
                  (C2 * m * (t_c - data[t])**m) * np.sin(omega * np.log(t_c - data[t])) +
                  (C2 * omega * (t_c - data[t])**m) * np.cos(omega * np.log(t_c - data[t]))
                 ) 
                for t in range(data.shape[0])
               ]).sum()
    
    grad_m = np.array([-(B * ((t_c - data[t])**m) * np.log(t_c - data[t]) +
                C1 * ((t_c - data[t])**m) * np.log(t_c - data[t]) * np.cos(np.log(t_c - data[t])) +
                C2 * ((t_c - data[t])**m) * np.log(t_c - data[t]) * np.sin(np.log(t_c - data[t]))
               )
             for t in range(data.shape[0])
             ]).sum()
    
    grad_omega = np.array([-(-C1 * ((t_c - data[t])**m) * np.sin(omega * np.log(t_c - data[t])) * np.log(t_c - data[t]) +
                    C2 * ((t_c - data[t])**m) * np.cos(omega * np.log(t_c - data[t])) * np.log(t_c - data[t])
                   )
                 for t in range(data.shape[0])
                 ]).sum()
    
    Jacobian = np.array((grad_t_c, grad_m, grad_omega))
    return Jacobian

In [101]:
Jacobian = Gradient(A = 5.7, B = -0.7, C1 = 0.5, C2 = 0.5, m = 0.68, omega = 6.5, t_c = 1999, data = np.array(data.time))
Jacobian

(2000.1504732274961, 11052.525953634515, -295.3939435950849)

### 4. Definition of Hessian and iterative step

In [95]:
def Hessian(Jacobian, mu):
    Jacobian = np.array(Jacobian).reshape((1,3))
    Hessian = np.linalg.inv((np.matmul(Jacobian.transpose(), Jacobian) + mu * np.identity(3))
                           )
    
    return Hessian

In [97]:
def step(t_c, m, omega, Jacobian, Hessian, cost):
    Jacobian = np.array(Jacobian).reshape((1,3))
    t_c = t_c - np.matmual(Hessian, Jacobian.transpose())[0] * cost
    m = m - np.matmual(Hessian, Jacobian.transpose())[1] * cost
    omega = omega - np.matmual(Hessian, Jacobian.transpose())[1] * cost
    return (t_c, m, omega)

### 5. Levenberg-Marquardt algorithm

In [ ]:
def LMA()